## Let's explore the Pipette and Camera objects

Let's start by importing the *tool modules* and our *Jubilee machine*. We'll also need an Opentrons OT-2 Pipette and a Raspberry Pi Camera V3 interfaced as a WebCamera tool. 
Both of these tools are outlined and defined in the `Science_Jubilee/tool_library` section of the [repository](https://github.com/machineagency/science_jubilee)

In [1]:
from science_jubilee import Machine as Jub
from science_jubilee.labware.Labware import Labware, Well
from science_jubilee.tools import Pipette, WebCamera, Tool

In [ ]:
1. Machine, Deck and Labware


Before we can start using our tools, we will need to make sure we are connected to our Jubilee and set up its deck, and define all the labware we will use for this notebook. 

If you are using your own laptop, you can use the machine's IP `address` to connect, otherwise if oyu are interfacing it wil a Raspberry Pi, you can use `address='duet3.local'` instead

In [ ]:
jubilee = Jub.Machine(address='192.168.1.2')

Before we can set up the `deck` of our `Jubilee` with all our labware, we will need to *home* **all** its axis.

> Attention -- !!! **BEFORE** !!! running the next cell, make sure that NO labware is loaded onto the deck or the Tool Changer will crush into them. 

In [ ]:
jubilee.home_all()

### Deck preparation and labware definition 
Awesome, now that we have our machine all ready to go, we will need to load all the labware onto it both *physically* and *digitally* . 

* Physically: place the desired/required labware for the Demo onto the deck of your Jubilee
* Digitally: first, we need to define our `deck`; then define and load the `labware` we want to use and indicate on which `slot` each of them was assigned to

In [ ]:
# ---------------- Deck -------------------
deck = jubilee.load_deck('lab_automation_deck.json')

Next, we will need to define our labware. In this case, you will need the following: 

* A **Tiprack** that is suitable for the pipette you are using to transfer your liquids
* A **Sample Labware** that will be used to collect the colors that your Rob Ross will make 
* A **Stock Labware** that will host the stock colors that your Rob Ross can mix together
* A **trash** to collect the used tips

In [ ]:
# -------------- Labware ------------------
tiprack = jubilee.load_labware('opentrons_96_tiprack_300ul.json', 0)
tiprack.load_manualOffset()
samples = jubilee.load_labware('fisherbrand_96_wellplate_360ul.json', 2)
samples.load_manualOffset()
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 3)
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

You can notice that the *tiprack* and the *samples* labware have a `manualOffset()` applied to them. This is to ensure that the labware coordinates are as precise as we can. The manual offset is **labware** and **slot** dependent, so you will need to run it prior to do so. You can find more information on how to run this in the `Science Jubilee` repository/documentation

### Tool import and set up

Once again we will need to load all our tools onto our machine both *physically* and *digitally* . 

To learn how to load the tools *physically*, you can find more information and resources in the `Science_Jubilee` [documentation](https://machineagency.github.io/science_jubilee/building/building_tools.html#building-tools)

Each tool will at minimum require a `tool_index`, an integer, and a `tool_name`. These should be the same as the ones defined in the machine's `config.g` file. 
Some tools will also required further information. You can defined these individually or in a config file for easier tool load. 

**Pipette Tool**

In [ ]:
P300 = Pipette.Pipette.from_config(1, 'Pipette', 'P300_config.json')
jubilee.load_tool(P300)

Even though you will be able to use te pipette after these two single lines, you can also associate its `tiprack` to the tool, as well as define a `trash` location. This will just make the code more clean.


In [ ]:
P300.add_tiprack(tiprack)
P300.trash = trash[0]

We can now pickup our tool and start "*playing*" around with it. Let's explore some of the functionalities of the `Pipette` tool.

In [2]:
jubilee.pickup_tool(P300)

# jubilee.pickup_tool(1) #These two statements are the same! why?

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\meryp\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\meryp\AppData\Local\Temp/ipykernel_17964/716732631.py", line 1, in <module>
    jubilee.pickup_tool(P300)
NameError: name 'jubilee' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\meryp\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\meryp\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "c:\Users\meryp\A

TypeError: object of type 'NoneType' has no len()

Now, that our tool is active, we can choose start moving liquids around. There are different way to do so. You can :

* pickup a tip
* aspirate a certain volume ( in $\mu$ L) from a source reservoir/well
* dispense it into a destination well
* return the tip ( if we want to reuse it) *OR* drop the tip in the trash

In [ ]:
P300.pickup_tip() 
# or P300.pickup_tip(tiprack['A1'])
# or P300.pickup_tip(tiprack[0])

All of the statement above are equivalent. Why?

In [ ]:
P300.aspirate(250, stocks['A1'].bottom(3))

In [ ]:
P300.dispense(250, samples[0].top(-1))

**Optional**

You can also add mixing steps either *before* or *after* or **both** your `dispense()` steps. For that you will need to indicate `the volume to mix` and `how many times`.

`P300.mix(200, 3)`

you can also change the speed by adding an `s` keyword argument. 
Try it below:

Once we are done, we can drop our tip into the trash or return it to its original location

In [ ]:
P300.drop_tip()

# or P300.drop_tip(trash[0]) 

    Why can we omit the location in where to drop/return the tip?

As we just saw, we can use the `Pipette.aspirate()` and `Pipette.dispense()` steps to take care of our liquid handling. However, we can also use a method called `transfer()` which will take care of both of them.

In [ ]:
P300.transfer(250, stocks[1].bottom(5), samples[1].top(-1), 
            blowout = True, new_tip='once',
            mix_after = (200, 3))

Take a look at all of the paramters and you'll see that you we have all of the same steps as before. 
We also have added a `blowout` step which will execute after the `dispense` step.

You can also indicate a list of volumes and source_wells to make things a bit easier. The index of volumes and source_wells are 1:1. 

In [ ]:
P300.transfer([50,100, 50], [stocks[0].bottom(5), stocks[2].bottom(5), stocks[1].bottom(5)],
            samples[1].top(-1), blowout = True, new_tip='once', mix_after = (200, 3, stocks[1]))

Take a look at what happend after it perfermed the mix step. When did that happen? Why was the tip dropped into the trash instead of the tiprack?

ok, we are now done with our Pipette short demo. Let's go and pick up the camera.

**Camera Tool**

In [ ]:
#Camera Tool
Camera = WebCamera.Camera.from_config(0, 'Camera', 'WebCamera_config.json')
jubilee.load_tool(Camera)

Awesome, we have now defined the (web)camera tool. Let's pick it up

In [ ]:
jubilee.pickup_tool(Camera)

Wait, our pipette was still attached to our Jubilee. What happened?

Ok, crisis averted!

Since I am a bit curious, let's actually start the Camera `video_feed` so that we can keep and eye on it

In [ ]:
Camera.video_feed()

 

We can now go and capture an image of our samples that we just made. To do so, we can just tell the camera to go `capture an image` of our well and view it

In [ ]:
my_image = Camera.capture_image(samples[1])
Camera.view_mage(my_image)

If your camera has a *ring light* on, you can also use it during your image captue.

In [ ]:
my_new_image = Camera.capture_image(samples[1], light=True, light_intensity = 1)
Camera.view_mage(my_new_image)

We can also obtaine the `RGB` values of our sample by processing our image. 

In [ ]:
Camera.process_image(my_image, radius = 50)

We can now park the camera:

In [ ]:
jubilee.park_tool()